In [2]:
# !pip install androguard==3.3.5
# !pip install tensorflow==2.13.0
# !pip install keras==2.13.1

In [4]:
import sys
from androguard.core.bytecodes import apk, dvm
from androguard.core.analysis import analysis
from androguard.core.bytecodes.apk import APK
import os
import re
sys.path.append("."+os.sep+'Log')
import logging as log

In [24]:
import numpy as np
import os
import logging
import sys
sys.path.append("."+os.sep+'GetFeature')
#import GetFeature
sys.path.append("."+os.sep+'Log')
#"from log import Log
import pandas as pd
import threading
import random

In [25]:
def getPermissions(Path, fileName,dataPath):
		"""
		get the API Calls
		:param Path: the APK folder path
		:param fileNmae: the APK Name
		:param dataPath: the folder path where the permission files save
		:return: permission files
		"""
		filePath = Path + os.sep + fileName
		print("Extract" + fileName + "Permissions")
		app = APK(filePath)
		permissions = app.get_permissions()
		name = fileName.replace(".apk", "")
		with open(dataPath+os.sep+name+"_Permission.txt",'w') as f:
			print("Saving" + fileName + "Permissions")
			for i in range(permissions.__len__()):
				f.write(permissions[i]+"\n")
    

In [26]:
def getAPICalls(Path, fileName, dataPath):
		"""
		get the API Calls
		:param Path: the APK folder path
		:param fileNmae: the APK Nmae
		:param dataPath: the folder path where the api files save
		:return: api files
		"""
		filePath = Path + os.sep + fileName
		# self.logger.info("Extract" + fileName + "API calls")
		print("Extract" + fileName + "API calls")
		app = APK(filePath)
		app_dex = dvm.DalvikVMFormat(app.get_dex())
		app_x = analysis.Analysis(app_dex)
		APIs = list()

		classes = [cc.get_name() for cc in app_dex.get_classes()]
		for method in app_dex.get_methods():
			methodBlock = app_x.get_method(method)
			if method.get_code() == None:
				continue
			for i in methodBlock.get_basic_blocks().get():
				for ins in i.get_instructions():
					output = ins.get_output()
					match = re.search(r'(L[^;]*;)->([^\(]*)', output)
					if match and match.group(1) not in classes:
						# print "API: "+match.group()+"	 "+match.group(1)
						if match.group(2) == "<init>":
							continue
						api = match.group()
						if api in APIs:
							continue
						else:
							APIs.append(api)

		name = fileName.replace(".apk", "")
		with open(dataPath+os.sep+name+"_API.txt",'w') as f:
			print("Saving " + fileName + " APIs")
			# self.logger.info("Saving " + fileName + " APIs")
			for i in range(APIs.__len__()):
					f.write(APIs[i]+"\n")

In [27]:
APIFile = "C:/Users/SRMAP/Desktop/FDP Code/Features/API.txt"
permissionFile = "C:/Users/SRMAP/Desktop/FDP Code/Features/permission.txt"
featurelistPath = "C:/Users/SRMAP/Desktop/FDP Code/Features/featureList.csv"
featurelist = list()

In [28]:
def featureFromFile(filePath):
		"""
		Get the features to be extracted from one feature file
		:param filePath:  file path
		:return: features list
		"""
		if not os.path.exists(filePath):
			#self.logger.error(filePath+"does not exist")
			print(filePath+"does not exist")
			return
		ls=list()
		with open(filePath,'r')as f:
			while(True):
				line=f.readline()
				if not line:break
				s=line.replace("\n","")
				s=s.replace("\r","")
				ls.append(s)
		return ls


In [29]:
def generateFeatureList():
		"""
		Get the features to be extracted from all features
		:param :
		:return: features list from all feature files
		"""
		ls1 = featureFromFile(APIFile)
		ls2 = featureFromFile(permissionFile)      
		featurelist = ls1+ls2

		frame = pd.DataFrame(featurelist)
		frame.to_csv(featurelistPath,header=False,index=False)

		return featurelist

In [30]:
featureList = generateFeatureList()

In [32]:
len(featureList)

158

# Malware Matrix

In [33]:
APKlist = list()
Matrix1 = np.zeros((0, len(featureList)), dtype="int")
label1 = list()

In [34]:
def featureFromFile(filePath):
		"""
		get features from file
		:param filePath: the feature files path
		:return: feature list
		"""
		if not os.path.exists(filePath):
			print(filePath+" does not exist")
			return
		ls=list()
		with open(filePath,'r',encoding='UTF-8')as f:
			while(True):
				line=f.readline()
				if not line:break
				s=line.replace("\n","")
				s=s.replace("\r","")
				ls.append(s)
		return ls


In [35]:
def getFeaturefromAPK(dataPath):
		apkFeature = np.zeros((1,featureList.__len__()), dtype=int)
		try:
			perList = featureFromFile(dataPath)
			for p in perList:
				if p in featureList:
					i = featureList.index(p)
					apkFeature[0][i] = 1
		except Exception as e:
			print("feature matrix extraction errors.")
		return apkFeature

In [36]:
import os
path = "C:/Users/SRMAP/Desktop/FDP Code/MalwareAPK/MalwareRawFeature"
dir_list = os.listdir(path)

In [37]:
for fl in dir_list:
    fl_path = path +'/'+ fl
    features = getFeaturefromAPK(fl_path)
    Matrix1 = np.vstack((Matrix1,features))
    label1.append(1)

feature matrix extraction errors.


In [38]:
len(Matrix1)

389

In [41]:
Matrix1

array([[0, 0, 0, ..., 0, 1, 0],
       [1, 1, 0, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [1, 1, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# Benign Matrix

In [42]:
featureList = generateFeatureList()
APKlist = list()
Matrix2 = np.zeros((0, len(featureList)), dtype="int")
label2 = list()

In [43]:
def featureFromFile(filePath):
		"""
		get features from file
		:param filePath: the feature files path
		:return: feature list
		"""
		if not os.path.exists(filePath):
			print(filePath+" does not exist")
			return
		ls=list()
		with open(filePath,'r',encoding='UTF-8')as f:
			while(True):
				line=f.readline()
				if not line:break
				s=line.replace("\n","")
				s=s.replace("\r","")
				ls.append(s)
		return ls


In [44]:
def getFeaturefromAPK(dataPath):
		apkFeature = np.zeros((1,featureList.__len__()), dtype=int)
		try:
			perList = featureFromFile(dataPath)
			for p in perList:
				if p in featureList:
					i = featureList.index(p)
					apkFeature[0][i] = 1
		except Exception as e:
			print("feature matrix extraction errors.")
		return apkFeature

In [45]:
import os
path = "C:/Users/SRMAP/Desktop/FDP Code/BenignAPK/BenignRawFeature"
dir_list = os.listdir(path)


In [46]:
for fl in dir_list:
    fl_path = path +'/'+ fl
    features = getFeaturefromAPK(fl_path)
    Matrix2 = np.vstack((Matrix2,features))
    label2.append(0)

feature matrix extraction errors.


In [47]:
len(Matrix2)

598

# Combine matrix

In [48]:
Matrix = np.vstack((Matrix1,Matrix2))

In [49]:
len(Matrix)

987

In [50]:
label = label1+label2

In [51]:
Matrix.shape

(987, 158)

# Model Training

In [54]:
import joblib
from sklearn.model_selection import train_test_split 
import tensorflow as tf
import os
import sys
import pickle 
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,Flatten,Multiply
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Input, Dense, concatenate
from keras.models import Model
from keras.models import load_model
from keras import optimizers

In [55]:
label = np.array(label)

In [56]:
dimemsion = 158
enc = OneHotEncoder()
label=label.reshape(-1, 1)
label=enc.fit_transform(label).toarray()

In [57]:
X_train,X_test,y_train,y_test=train_test_split(Matrix,label,shuffle=True,train_size=0.85)

In [58]:
def Att(att_dim,inputs,name):
		""" attention layer """	
		V = inputs
		QK = Dense(att_dim)(inputs)
		QK = Activation("softmax",name=name)(QK)
		MV = Multiply()([V, QK])
		return(MV)

In [59]:
inputs = Input(shape=(dimemsion,))
attention = Att(dimemsion,inputs,"attention")
output = Dense(2, activation='softmax')(attention)
model = Model(inputs=[inputs], outputs=output)

In [60]:
adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit([X_train], y_train, epochs=30, batch_size=16, shuffle=True, validation_split=0.1)

Epoch 1/30
48/48 [==============================] - 2s 10ms/step - loss: 0.6891 - accuracy: 0.6273 - val_loss: 0.6840 - val_accuracy: 0.5833
Epoch 2/30
48/48 [==============================] - 0s 4ms/step - loss: 0.6689 - accuracy: 0.6538 - val_loss: 0.6599 - val_accuracy: 0.6190
Epoch 3/30
48/48 [==============================] - 0s 3ms/step - loss: 0.6360 - accuracy: 0.7387 - val_loss: 0.6270 - val_accuracy: 0.7500
Epoch 4/30
48/48 [==============================] - 0s 3ms/step - loss: 0.5985 - accuracy: 0.8408 - val_loss: 0.5903 - val_accuracy: 0.8214
Epoch 5/30
48/48 [==============================] - 0s 3ms/step - loss: 0.5640 - accuracy: 0.8687 - val_loss: 0.5613 - val_accuracy: 0.8333
Epoch 6/30
48/48 [==============================] - 0s 3ms/step - loss: 0.5362 - accuracy: 0.8806 - val_loss: 0.5327 - val_accuracy: 0.8810
Epoch 7/30
48/48 [==============================] - 0s 4ms/step - loss: 0.5146 - accuracy: 0.8806 - val_loss: 0.5132 - val_accuracy: 0.8810
Epoch 8/30
48/48 [=

In [61]:
def show_accuracy(x_train,y_hat,y_train,name):
		"""
		calculate the training results
		:param x_train: feature matrix of training set 
		:param y_hat: detection result of training set 
		:param y_train: label of training set 
		:param name: apk name
		:return: Accurary 
		"""	

		print(name)
		tp = 0				  
		fp = 0
		tn = 0
		fn = 0
		num = len(x_train)			 
		for i in range(num):		 
			if y_hat[i] == y_train[i]: 
				if y_hat[i] == 0:
					tp += 1
				else:
					tn += 1
			else:
				if y_hat[i] == 0:
					fp += 1
				else:
					fn += 1
		Recall = tp / float(tp + fn)
		Precision = tp / float(tp + fp)
		Accurary = (tp + tn)/float(tp + tn + fn + fp)
		fmeasure = (2*Precision*Recall)/float(Precision+Recall)
		FPR = fp/float(tn + fp)       
		FalseRate = 1 - Precision
		AllFalseRate = 1- Accurary
		print("Total num:",num)
		printlist = ['Accurary','Precision','Recall','F1-Score','TPR','FPR']
		datalist = [Accurary,Precision,Recall,fmeasure,Recall,FPR]
		print(printlist)
		print(datalist)


In [62]:
y_hat_proba = model.predict(X_train)
y_hat = [round(i[1]) for i in y_hat_proba]
show_accuracy(X_train,y_hat,y_train[:,1],"train set")

27/27 [==============================] - 0s 2ms/step
train set
Total num: 838
['Accurary', 'Precision', 'Recall', 'F1-Score', 'TPR', 'FPR']
[0.9486873508353222, 0.9340866290018832, 0.9841269841269841, 0.9584541062801932, 0.9841269841269841, 0.10479041916167664]


In [63]:
y_hat_proba = model.predict(X_test)
y_hat = [round(i[1]) for i in y_hat_proba]
show_accuracy(X_test,y_hat,y_test[:,1],"test set")

5/5 [==============================] - 0s 3ms/step
test set
Total num: 149
['Accurary', 'Precision', 'Recall', 'F1-Score', 'TPR', 'FPR']
[0.8993288590604027, 0.8691588785046729, 0.9893617021276596, 0.9253731343283582, 0.9893617021276596, 0.2545454545454545]


# Testing

In [72]:
def getFeatureMatric(checkApks):
		"""
		get feature files from CheckAPK file
		:param checkApks: the checkApks files path
		:return: Matrix 
		"""		
		Matrix = np.zeros((0, self.Featurelist.__len__()), dtype=int)
		GetFeatureMatrixClass = GetFeatureMatrix()
		for apk in checkApks:
			features = GetFeatureMatrixClass.getFeaturefromAPK(CheckData,apk)
			Matrix = np.vstack((Matrix,features))
		return Matrix

In [73]:
featureList = generateFeatureList()
APKlist_T = list()
Matrix_test = np.zeros((0, len(featureList)), dtype="int")

In [74]:
Matrix_test

array([], shape=(0, 158), dtype=int32)

In [75]:
# C:\Users\91812\Desktop\FDP Presentation\Mitre Code\Ransomware-APKs\Koler_raw

test_file = "C:/Users/SRMAP/Desktop/FDP Code/Ransomware-APKs/Svpeng_raw/237d0253fea2a777798360175fd941e9_API_Permission.txt"
features = getFeaturefromAPK(test_file)

In [76]:
features

array([[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 1, 0, 1]])

In [77]:
Matrix_test = np.vstack((Matrix_test,features))

In [78]:
Matrix_test

array([[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 1, 0, 1]])

In [79]:
model.save('model.h5')

C:\Users\SRMAP\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [80]:
model = tf.keras.models.load_model('model.h5')
model.predict(Matrix_test)

1/1 [==============================] - 0s 83ms/step


array([[0.04574843, 0.9542516 ]], dtype=float32)